In [ ]:
!pip install nerus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ipymarkup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26114 sha256=326b588c599dcc366fd3e72a16e049dd9d754dc2a5364e60e0ae5a7857e81d95
  Stored in directory: /root/.cache/pip/wheels/ab/fa/1b/75d9a713279796785711bd0bad8334aaace560c0bd28830c8c
Successfully built intervaltree


In [ ]:
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00


In [ ]:
from nerus import load_nerus
import sklearn_crfsuite
from sklearn_crfsuite import metrics as crf_metrics
from ipymarkup import show_span_ascii_markup, show_dep_ascii_markup
# import ru_core_news_sm
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import wordpunct_tokenize
import nltk
from sklearn.model_selection import train_test_split
import random
nltk.download('punkt')
tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Nerus

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!gunzip -c gdrive/MyDrive/nerus_lenta.conllu.gz | head

# newdoc id = 0
# sent_id = 0_0
# text = Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости.
1	Вице-премьер	_	NOUN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	7	nsubj	_	Tag=O
2	по	_	ADP	_	_	4	case	_	Tag=O
3	социальным	_	ADJ	_	Case=Dat|Degree=Pos|Number=Plur	4	amod	_	Tag=O
4	вопросам	_	NOUN	_	Animacy=Inan|Case=Dat|Gender=Masc|Number=Plur	1	nmod	_	Tag=O
5	Татьяна	_	PROPN	_	Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing	1	appos	_	Tag=B-PER
6	Голикова	_	PROPN	_	Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing	5	flat:name	_	Tag=I-PER
7	рассказала	_	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	Tag=O


In [ ]:
docs = load_nerus('gdrive/MyDrive/nerus_lenta.conllu.gz')

# Ozon

In [ ]:
!wget dataset_ozon.tsv https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv

--2023-03-23 17:30:52--  http://dataset_ozon.tsv/
Resolving dataset_ozon.tsv (dataset_ozon.tsv)... failed: Name or service not known.
wget: unable to resolve host address ‘dataset_ozon.tsv’
--2023-03-23 17:30:52--  https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9640627 (9.2M) [text/plain]
Saving to: ‘dataset_ozon.tsv’

dataset_ozon.tsv    100%[===================>]   9.19M  --.-KB/s    in 0.09s   

2023-03-23 17:30:53 (98.7 MB/s) - ‘dataset_ozon.tsv’ saved [9640627/9640627]

FINISHED --2023-03-23 17:30:53--
Total wall clock time: 0.5s
Downloaded: 1 files, 9.2M in 0.09s (98.7 MB/s)


In [ ]:
!python3 -m spacy download ru_core_news_sm

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-23 17:30:58.498990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 17:31:03.285531: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-23 17:31:03.285692: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

In [ ]:
ds = pd.read_csv('/content/dataset_ozon.tsv', sep = '\t')
ds = ds.drop(columns = ['Unnamed: 0'])

In [ ]:
ds

,name,brand
0,"Игровая консоль PlayStation 5, белый",PlayStation
1,Золотой Шелк Маска для волос гиалуроновая кисл...,Золотой Шелк
2,"Игровая консоль Microsoft Xbox Series X, черный",Microsoft
3,L'Oreal Paris Infaillible 24H Fresh Wear Пудра...,L'Oreal Paris
4,Maybelline New York Lifter Gloss Блеск для губ...,Maybelline New York
...,...,...
81774,"Erborian BB Family BB карандаш, светлый, 3 г",Erborian
81775,Комплект модулей сменных фильтрующих Аквафор Р...,Аквафор
81776,Набор менструальных чаш Satisfyer Feel secure ...,Satisfyer
81777,28048 Подстилка для надувных и каркасных бассе...,Intex


In [ ]:
def make_tags(n, b):
    global tags
    ner_tags_id = []
    name = wordpunct_tokenize(n)
    brand = wordpunct_tokenize(b)
    results=[]
    sll=len(brand)
    for ind in (i for i,e in enumerate(name) if e==brand[0]):
        if name[ind:ind+sll]==brand:
            results.append((ind,ind+sll-1))
    if results != []:
      ner_tags_labels = ['O'] * len(name)  
      ner_tags_id = [0] * len(name)     
      for result in results:
          if result[0] == result[1]:
              ner_tags_labels[result[0]] = 'B-BRAND'
              ner_tags_id[result[0]] = 1
          else:
            ner_tags_labels[result[0]] = 'B-BRAND'
            ner_tags_id[result[0]] = 1
            for j in range(result[0]+1, result[1]+1):
              ner_tags_labels[j] = 'I-BRAND'
              ner_tags_id[j] = 2

      tags.append({'tokens': name, 'ner_tags_labels': ner_tags_labels, 'ner_tags_id': ner_tags_id})

In [ ]:
# nlp = ru_core_news_sm.load()

NameError: ignored

In [ ]:
tags = []
x = 0
ds.progress_apply(lambda x: make_tags(x['name'], x['brand']), axis = 1)

100%|██████████| 81779/81779 [00:11<00:00, 6851.44it/s]


0        None
1        None
2        None
3        None
4        None
         ... 
81774    None
81775    None
81776    None
81777    None
81778    None
Length: 81779, dtype: object

# Формируем данные для CRF

In [ ]:
full_list = []
for dictionary in tqdm(tags):
    lil_list = []
    for idx, element in enumerate(dictionary['tokens']):
        lil_list.append({
            'text': element,
            'bio-tag': dictionary['ner_tags_labels'][idx]
        })
    full_list.append(lil_list)

100%|██████████| 81527/81527 [00:03<00:00, 23788.63it/s]


In [ ]:
full_list

[[{'text': 'Игровая', 'bio-tag': 'O'},
  {'text': 'консоль', 'bio-tag': 'O'},
  {'text': 'PlayStation', 'bio-tag': 'B-BRAND'},
  {'text': '5', 'bio-tag': 'O'},
  {'text': ',', 'bio-tag': 'O'},
  {'text': 'белый', 'bio-tag': 'O'}],
 [{'text': 'Золотой', 'bio-tag': 'B-BRAND'},
  {'text': 'Шелк', 'bio-tag': 'I-BRAND'},
  {'text': 'Маска', 'bio-tag': 'O'},
  {'text': 'для', 'bio-tag': 'O'},
  {'text': 'волос', 'bio-tag': 'O'},
  {'text': 'гиалуроновая', 'bio-tag': 'O'},
  {'text': 'кислота', 'bio-tag': 'O'},
  {'text': 'и', 'bio-tag': 'O'},
  {'text': 'коллаген', 'bio-tag': 'O'},
  {'text': '"', 'bio-tag': 'O'},
  {'text': 'Реанимация', 'bio-tag': 'O'},
  {'text': 'волос', 'bio-tag': 'O'},
  {'text': 'Керапластика', 'bio-tag': 'O'},
  {'text': '",', 'bio-tag': 'O'},
  {'text': '180', 'bio-tag': 'O'},
  {'text': 'мл', 'bio-tag': 'O'}],
 [{'text': 'Игровая', 'bio-tag': 'O'},
  {'text': 'консоль', 'bio-tag': 'O'},
  {'text': 'Microsoft', 'bio-tag': 'B-BRAND'},
  {'text': 'Xbox', 'bio-tag': 'O

In [ ]:
len(tags)

81527

In [ ]:
len(full_list)

81527

In [ ]:
count = 0
for doc in tqdm(docs):
    if count >= len(tags):
        break
    for sent in doc.sents:
        lil_list = []
        for token in sent.tokens:
            lil_list.append({
                'text': token.text,
                'bio-tag': token.tag
            })
        full_list.append(lil_list)
        count += 1

6876it [00:27, 250.50it/s]


In [ ]:
random.shuffle(full_list)

In [ ]:
train_sents, test_sents = train_test_split(full_list)

In [ ]:
train_sents[0]

[{'text': 'Презервативы', 'bio-tag': 'O'},
 {'text': 'DUETT', 'bio-tag': 'B-BRAND'},
 {'text': 'classic', 'bio-tag': 'O'},
 {'text': '/', 'bio-tag': 'O'},
 {'text': 'классика', 'bio-tag': 'O'},
 {'text': 'выгодная', 'bio-tag': 'O'},
 {'text': 'упаковка', 'bio-tag': 'O'},
 {'text': '30', 'bio-tag': 'O'},
 {'text': 'шт', 'bio-tag': 'O'},
 {'text': '.', 'bio-tag': 'O'}]

In [ ]:
test_sents[0]

[{'text': 'Мяч', 'bio-tag': 'O'},
 {'text': 'баскетбольный', 'bio-tag': 'O'},
 {'text': 'Nike', 'bio-tag': 'B-BRAND'},
 {'text': 'Versa', 'bio-tag': 'O'},
 {'text': 'Tack', 'bio-tag': 'O'},
 {'text': '8P', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': 'янтарный', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': 'черный', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': 'серебристый', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': 'размер', 'bio-tag': 'O'},
 {'text': '6', 'bio-tag': 'O'}]

In [ ]:
def word2features(sent, i):
    word = sent[i]['text']
    
    features = {
        'text': word
    }
    if i > 0:
        word1 = sent[i-1]['text']
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1]['text']
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [word['bio-tag'] for word in sent]

def sent2tokens(sent):
    return [word['text'] for word in sent]

In [ ]:
X_train = [sent2features(text) for text in train_sents]
y_train = [sent2labels(text) for text in train_sents]

X_test = [sent2features(text) for text in test_sents]
y_test = [sent2labels(text) for text in test_sents]

In [ ]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass
y_pred  = crf.predict(X_test)

In [ ]:
labels = set()
for y_big in y_train:
  for y_small in y_big:
    labels.add(y_small)

labels = list(labels)

In [ ]:
crf_metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.979617027407851

In [ ]:
crf_metrics.flat_precision_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9799589739802225

In [ ]:
crf_metrics.flat_recall_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9802978856963506